In [2]:
import chromedriver_binary
from selenium import webdriver
from time import sleep
import openpyxl
import pandas as pd
import unicodedata
import re
import datetime
from sucreiping import Sucreiping
from scraping_detail_data import Sucraping_detail

In [2]:
sucreiping = Sucreiping()
df = sucreiping.get_data()

In [3]:
detail = Sucraping_detail()
detail_df = detail.get_detail_data()

In [4]:
df.to_csv("cow_data.csv")

In [5]:
detail_df.to_csv("cow_detail_data.csv")

In [3]:
df = pd.read_csv("cow_data.csv")

In [4]:
detail_df = pd.read_csv("cow_detail_data.csv")

In [5]:
detail_df = detail_df.rename(columns={'母の祖々父': '母の祖祖父'})

In [6]:
df = df.rename(columns={'Unnamed: 0': 'index'})
detail_df = detail_df.rename(columns={'Unnamed: 0': 'index'})

In [7]:
df["年月日"] = pd.to_datetime(df["年月日"])

In [8]:
detail_df["年月日"] = pd.to_datetime(detail_df["年月日"])

In [12]:
df.columns[6:9]

Index(['日令', '体重', '価格'], dtype='object')

In [13]:
column = df.columns[6:9]

for col in column:
    df[col] = df[col].astype('int')

In [17]:
test_df = pd.DataFrame()
for i in range(len(detail_df)):
    search_df = df[(df["父牛"]==detail_df["父牛"][i]) & (df["母の父"]==detail_df["母の父"][i]) & (df["母の祖父"]==detail_df["母の祖父"][i]) & (df["母の祖祖父"]==detail_df["母の祖祖父"][i]) & (df["日令"]==detail_df["日令"][i]) & (df["性別"]==detail_df["性別"][i]) & (df["年月日"]==detail_df["年月日"][i])]
    
    test_df = pd.concat([test_df,search_df])
test_df=test_df.reset_index(drop=True)

In [18]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26579 entries, 0 to 26578
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   26579 non-null  int64         
 1   性別      26579 non-null  object        
 2   父牛      26579 non-null  object        
 3   母の父     26579 non-null  object        
 4   母の祖父    26579 non-null  object        
 5   母の祖祖父   26579 non-null  object        
 6   日令      26579 non-null  int64         
 7   体重      26579 non-null  int64         
 8   価格      26579 non-null  int64         
 9   年月日     26579 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 2.0+ MB


In [19]:
_df = pd.DataFrame()
for i in range(len(df)):
    search_df = detail_df[(df["父牛"][i]==detail_df["父牛"]) & (df["母の父"][i]==detail_df["母の父"]) & (df["母の祖父"][i]==detail_df["母の祖父"]) & (df["母の祖祖父"][i]==detail_df["母の祖祖父"]) & (df["日令"][i]==detail_df["日令"]) & (df["性別"][i]==detail_df["性別"]) & (df["年月日"][i]==detail_df["年月日"])]
    
    _df = pd.concat([_df,search_df])
_df=_df.reset_index(drop=True)

In [20]:
last_df = pd.concat([test_df,_df],axis=1)

In [21]:
last_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26579 entries, 0 to 26578
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   26579 non-null  int64         
 1   性別      26579 non-null  object        
 2   父牛      26579 non-null  object        
 3   母の父     26579 non-null  object        
 4   母の祖父    26579 non-null  object        
 5   母の祖祖父   26579 non-null  object        
 6   日令      26579 non-null  int64         
 7   体重      26579 non-null  int64         
 8   価格      26579 non-null  int64         
 9   年月日     26579 non-null  datetime64[ns]
 10  index   26579 non-null  int64         
 11  入場      26579 non-null  int64         
 12  品種      26579 non-null  object        
 13  名号      26431 non-null  object        
 14  期待育種価   15372 non-null  object        
 15  耳標番号    26232 non-null  object        
 16  資格      26579 non-null  object        
 17  性別      26579 non-null  object        
 18  生年月日  

In [22]:
last_df.head()

,index,性別,父牛,母の父,母の祖父,母の祖祖父,日令,体重,価格,年月日,...,母牛生年,産次,備考1,備考2,生産者住所,氏名,主要飼料名1,主要飼料名2,主要飼料名3,年月日
0,397,去勢,福之姫,百合白清２,安福久,平茂勝,278,299,891000,2022-04-07,...,2008-01-08 00:00:00,11.0,リング去勢,NaN,玉名郡和水町,南里文章,ブリーズ,,,2022-04-07
1,372,雌,福之姫,光平照,平茂幸,谷川,291,233,583000,2022-04-07,...,2016-01-08 00:00:00,3.0,NaN,NaN,阿蘇郡産山村,井 星二,ダイヤＭ,阿畜１号,,2022-04-07
2,363,雌,福之姫,安福勝,茂勝栄,鶴長,271,284,711700,2022-04-07,...,2019-05-01 00:00:00,1.0,左腿刺毛,NaN,菊池郡菊陽町,㈱フジモトファーム,ハイグロＭＯ,,,2022-04-07
3,296,雌,美津照重,福之姫,安茂勝,福谷福,297,284,715000,2022-04-07,...,2019-05-01 00:00:00,1.0,NaN,NaN,菊池郡菊陽町,㈱フジモトファーム,ハイグロＭＯ,,,2022-04-07
4,408,去勢,福之姫,隆之国,安福久,百合茂,252,296,902000,2022-04-07,...,2018-01-08 00:00:00,2.0,NaN,NaN,菊池郡菊陽町,小平利徳,熊畜グローエース,新育成用ぐんぐんＡ,,2022-04-07
